# Importing Libraries

In [1]:
!pip install cudf-cu12 dask-cudf-cu12 cupy-cuda12x --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [1]:
import os
import zipfile

## GPU Detection

In [2]:
import subprocess
try:
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'])
    print("Detected GPU!!!")
    GPU_AVAILABLE = True
except Exception as e:
    print(f'No GPU Detected. Running on CPU!\n{e}')
    GPU_AVIALABLE = False

Detected GPU!!!


In [3]:
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 30.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34065,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 30.00 GiB
Comm: tcp://127.0.0.1:39677,Total threads: 1
Dashboard: http://127.0.0.1:43917/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:38499,


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [5]:
try:
    import cudf
    import cupy as cp
    import dask_cudf
    GPU_MODE = torch.cuda.is_available()
except ImportError:
    cudf = None
    cp = None
    dask_cudf = None
    GPU_MODE = False

In [6]:
try:
    torch.multiprocessing.set_start_method('spawn', force=True)
    print("Set multiprocessing start method to 'spawn'.")
except RuntimeError:
    pass

Set multiprocessing start method to 'spawn'.


In [7]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    confusion_matrix
)
from tqdm.auto import tqdm

# Configuration Settings

In [8]:
train_dir = "/kaggle/input/magnetic-dataset/magnetic_data/train"
dev_dir = "/kaggle/input/magnetic-dataset/magnetic_data/dev"
test_dir = "/kaggle/input/magnetic-dataset/magnetic_data/test"
scaler_params_path = "scaler_params.csv"

# CLIENT = Client(n_workers=4, threads_per_worker=2, memory_limit='4GB')
# print(f"Dask Dashboard Link: {CLIENT.dashboard_link}\n")

features = [
    'USFLUX_log', 'TOTPOT_log', 'PIL_LEN_log', 'MEANSHR', 'TOTFZ', 'EPSZ', 'R_VALUE',
    'USFLUX_log_roll_mean5', 'USFLUX_log_roll_std5', 'TOTPOT_log_diff1', 'MEANSHR_lag3'
]
target = 'label'
class_names = ['No Flare', 'C-Class', 'M-Class', 'X-Class']

batch_size = 256

num_cpus = os.cpu_count()
pin_memory = True
load_pre_trained_model = False


# Importing the Dataset

Corrects a flawed Dask DataFrame by keeping only the records that
match the documented, correct sequence length.

Args:
    ddf (dd.DataFrame): The input Dask DataFrame with variable length sequences.
    desired_length (int): The desired, correct sequence length.

Returns:
    dd.DataFrame: A new, cleaned Dask DataFrame.

In [9]:
def filter_by_sequence_length(ddf, desired_length=60):
    print(f"\n--- Applying Data Correction: Filtering for sequence length == {desired_length} ---")
    
    # Get the number of rows for each record_id
    print("   - Calculating sequence lengths for each record...")
    sequence_lengths = ddf.groupby('record_id').record_id.count().compute()
    
    # Find the record_ids that have the correct length
    ids_to_keep = sequence_lengths[sequence_lengths == desired_length].index
    
    num_original = len(sequence_lengths)
    num_to_keep = len(ids_to_keep)
    num_to_discard = num_original - num_to_keep
    
    print(f"   - Found {num_original} total unique records.")
    print(f"   - Keeping {num_to_keep} records with the correct length.")
    print(f"   - Discarding {num_to_discard} records with incorrect lengths.")
    
    # Filter the original DataFrame to create the cleaned version
    cleaned_ddf = ddf[ddf['record_id'].isin(ids_to_keep)]
    
    print("--- Data Correction Complete ---")
    return cleaned_ddf

In [10]:
def encode_target(ddf, target_col_name):
    meta_dict = {col: ddf[col].dtype for col in ddf.columns if col != target_col_name}
    meta_dict['flare_level'] = np.int8
    
    def encode_partition(df):
        df['flare_level'] = 0
        df.loc[df[target_col_name].str.startswith('C'), 'flare_level'] = 1
        df.loc[df[target_col_name].str.startswith('M'), 'flare_level'] = 2
        df.loc[df[target_col_name].str.startswith('X'), 'flare_level'] = 3
        df['flare_level'] = df['flare_level'].astype(np.int8)
        return df.drop(columns=[target_col_name])
    
    meta_df = pd.DataFrame({col: pd.Series(dtype=dtype) for col, dtype in meta_dict.items()})
    
    ddf = ddf.map_partitions(encode_partition, meta=meta_df)
    
    return ddf

In [11]:
def Index(ddf):
    ddf["_safe_index"] = ddf["record_id"]*60 + ddf["seq_id"]
    ddf = ddf.map_partitions(lambda df: df.sort_values("_safe_index"))
    ddf = ddf.set_index("_safe_index", sorted=False, compute=True)
    return ddf

In [12]:
# train_dir = "../data/processed/magnetic_data/train/"
ddf_train = dask_cudf.read_parquet(train_dir)
ddf_train = filter_by_sequence_length(ddf_train, desired_length=60)
# if 'index' in ddf_train.columns:
#     ddf_train = ddf_train.drop(columns=['index'])
ddf_train = Index(ddf_train)
ddf_train = encode_target(ddf_train, target)
print(f"Indexed ddf_train: {ddf_train.index.is_monotonic_increasing.compute()}")


--- Applying Data Correction: Filtering for sequence length == 60 ---
   - Calculating sequence lengths for each record...
   - Found 167122 total unique records.
   - Keeping 125459 records with the correct length.
   - Discarding 41663 records with incorrect lengths.
--- Data Correction Complete ---


2025-11-08 15:37:14,810 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 initialized by task ('shuffle-transfer-b00adfbeb0fcf232b360e8fb272f3528', 3) executed on worker tcp://127.0.0.1:45783
2025-11-08 15:37:22,466 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 deactivated due to stimulus 'task-finished-1762616242.4650664'


Indexed ddf_train: True


In [13]:
# train_dir = "../data/processed/magnetic_data/train/"
ddf_dev = dask_cudf.read_parquet(dev_dir)
ddf_dev = filter_by_sequence_length(ddf_dev, desired_length=60)
ddf_dev = Index(ddf_dev)
ddf_dev = encode_target(ddf_dev, target)
print(f"Indexed ddf_dev: {ddf_dev.index.is_monotonic_increasing.compute()}")

# X_train = ddf_train[features].to_dask_array().compute_chunk_sizes()
# y_train = ddf_train['flare_level'].to_dask_array().compute_chunk_sizes()


--- Applying Data Correction: Filtering for sequence length == 60 ---
   - Calculating sequence lengths for each record...
   - Found 1706 total unique records.
   - Keeping 1276 records with the correct length.
   - Discarding 430 records with incorrect lengths.
--- Data Correction Complete ---


2025-11-08 15:37:34,772 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 49c4e8895e8e87d16d28e7b3d8b5f9ef initialized by task ('shuffle-transfer-49c4e8895e8e87d16d28e7b3d8b5f9ef', 3) executed on worker tcp://127.0.0.1:45783
2025-11-08 15:37:35,119 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 49c4e8895e8e87d16d28e7b3d8b5f9ef deactivated due to stimulus 'task-finished-1762616255.1181455'


Indexed ddf_dev: True


In [14]:
# test_dir = "../data/processed/magnetic_data/test/"
ddf_test = dask_cudf.read_parquet(test_dir)
ddf_test = filter_by_sequence_length(ddf_test, desired_length=60)
ddf_test = Index(ddf_test)
ddf_test = encode_target(ddf_test, target)
print(f"Indexed ddf_test: {ddf_test.index.is_monotonic_increasing.compute()}")

# X_test = ddf_test[features].to_dask_array().compute_chunk_sizes()
# y_test = ddf_test['flare_level'].to_dask_array().compute_chunk_sizes()


--- Applying Data Correction: Filtering for sequence length == 60 ---
   - Calculating sequence lengths for each record...
   - Found 1706 total unique records.
   - Keeping 1271 records with the correct length.
   - Discarding 435 records with incorrect lengths.
--- Data Correction Complete ---


2025-11-08 15:37:38,847 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1f9732e8467cb3f9b818c221039d14f5 initialized by task ('shuffle-transfer-1f9732e8467cb3f9b818c221039d14f5', 3) executed on worker tcp://127.0.0.1:39677
2025-11-08 15:37:39,045 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1f9732e8467cb3f9b818c221039d14f5 deactivated due to stimulus 'task-finished-1762616259.0435302'


Indexed ddf_test: True


In [15]:
df_gpu = ddf_train[['flare_level']].compute()  # Bring to a single CuDF on GPU
class_counts_multi = df_gpu['flare_level'].value_counts()
print(f"Multiclass Distribution:\n{class_counts_multi}")
print("Note the mapping: 0=No Flare, 1=C-Class, 2=M-Class, 3=X-Class\n")

2025-11-08 15:37:42,756 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 initialized by task ('shuffle-transfer-b00adfbeb0fcf232b360e8fb272f3528', 3) executed on worker tcp://127.0.0.1:45783
2025-11-08 15:37:49,700 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 deactivated due to stimulus 'task-finished-1762616269.6992314'


Multiclass Distribution:
flare_level
0    6730680
1     670380
2     113640
3      12840
Name: count, dtype: int64
Note the mapping: 0=No Flare, 1=C-Class, 2=M-Class, 3=X-Class



# Feature Scaling and Normalization

In [16]:
aggregations = {col: ['mean', 'std'] for col in features if col not in ['record_id', 'seq_id']}
scaler_params_df = (
    ddf_train.map_partitions(
        lambda gdf: gdf.groupby("seq_id").agg(aggregations)
    ).compute()
)

scaler_params_df.columns = ['_'.join(col).strip() for col in scaler_params_df.columns.values]
scaler_params_df.to_csv(scaler_params_path)

2025-11-08 15:37:50,630 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 initialized by task ('shuffle-transfer-b00adfbeb0fcf232b360e8fb272f3528', 0) executed on worker tcp://127.0.0.1:45783
2025-11-08 15:37:57,564 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 deactivated due to stimulus 'task-finished-1762616277.5632808'


In [17]:
def timeseries_scaling(ddf, scaler_params_path, features):
    if isinstance(ddf, dd.DataFrame):
        ddf = dask_cudf.from_cudf(ddf.compute(), npartitions=ddf.npartitions)

    scaler_params = cudf.read_csv(scaler_params_path)
    scaler_params_ddf = dask_cudf.from_cudf(scaler_params, npartitions=1)

    ddf_scaled = ddf.merge(scaler_params_ddf, on="seq_id", how="left")

    for col in features:
        mean_col = f"{col}_mean"
        std_col = f"{col}_std"
        ddf_scaled[col] = (ddf_scaled[col] - ddf_scaled[mean_col]) / (ddf_scaled[std_col] + 1e-7)

    return ddf_scaled.persist()

In [18]:
ddf_train = timeseries_scaling(ddf_train, scaler_params_path, features).persist()
ddf_dev = timeseries_scaling(ddf_dev, scaler_params_path, features).persist()
ddf_test = timeseries_scaling(ddf_test, scaler_params_path, features).persist()
# ddf_train.head(100)

2025-11-08 15:37:58,548 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 initialized by task ('shuffle-transfer-b00adfbeb0fcf232b360e8fb272f3528', 0) executed on worker tcp://127.0.0.1:45783
2025-11-08 15:38:05,031 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle b00adfbeb0fcf232b360e8fb272f3528 deactivated due to stimulus 'task-finished-1762616285.0306206'
/usr/local/lib/python3.11/dist-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 811.44 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-11-08 15:38:18,967 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 49c4e8895e8e87d16d28e7b3d8b5f9ef initialized by task ('shuffle-transfer-49c4e8895e8e87d16d28e7b

In [20]:
!nvidia-smi

Sat Nov  8 11:00:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             32W /   70W |    5585MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# DataLoader

In [19]:
class TimeSeriesDataset(Dataset):
    def __init__(self, ddf, features=features, target="flare_level", chunk_size=10000):
        print(f"\n{'='*60}")
        print(f"Loading TimeSeriesDataset (chunk size: {chunk_size})...")
        print(f"{'='*60}")
        
        self.features_col = features
        self.target = target
        seq_length = 60
        num_features = len(features)
        
        unique_ids_series = ddf["record_id"].unique().compute()
        
        if hasattr(unique_ids_series, 'to_arrow'):
            record_list = unique_ids_series.to_arrow().to_pylist()
        elif hasattr(unique_ids_series, 'to_pandas'):
            record_list = unique_ids_series.to_pandas().tolist()
        else:
            record_list = list(unique_ids_series)
        
        num_records = len(record_list)
        print(f"\tFound {num_records} unique records")
        
        self.X = []
        self.y = []
        
        num_chunks = (num_records + chunk_size - 1) // chunk_size
        
        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * chunk_size
            end_idx = min(start_idx + chunk_size, num_records)
            chunk_records = record_list[start_idx:end_idx]
            
            print(f"\n\tChunk {chunk_idx+1}/{num_chunks}: Records {start_idx}-{end_idx}")
            
            try:
                chunk_ddf = ddf[ddf["record_id"].isin(chunk_records)]
                chunk_df = chunk_ddf.compute()
                
                if hasattr(chunk_df, 'to_pandas'):
                    chunk_df = chunk_df.to_pandas()
                
                print(f"\t\tLoaded {len(chunk_df)} rows")
                
                # Group by record_id and process
                grouped = chunk_df.groupby("record_id")
                
                for record_id, seq_df in grouped:
                    actual_len = len(seq_df)
                    
                    # Debug first few mismatches
                    if actual_len != seq_length and len(self.X) < 5:
                        print(f"\t\tRecord {record_id}: {actual_len} rows (expected {seq_length})")
                    
                    # Take first 60 rows if more
                    if actual_len >= seq_length:
                        seq_df = seq_df.iloc[:seq_length]
                    else:
                        continue
                    
                    try:
                        features_array = seq_df[self.features_col].values.astype(np.float32)
                        target_value = seq_df[self.target].mode().iloc[0]
                        
                        self.X.append(features_array)
                        self.y.append(target_value)
                        
                    except Exception as e:
                        if len(self.X) < 3:
                            print(f"\t\t\tERROR: {e}")
                        continue
                
                print(f"\t\tTotal loaded: {len(self.X)} sequences")
                
                del chunk_df, chunk_ddf
                
            except Exception as e:
                print(f"\t\tCHUNK ERROR: {e}")
                continue
        
        # Convert to numpy
        if len(self.X) == 0:
            raise ValueError("No sequences loaded! Check preprocessing pipeline.")
        
        self.X = np.array(self.X, dtype=np.float32)
        self.y = np.array(self.y, dtype=np.int64)
        
        print(f"\n{'='*60}")
        print(f"Dataset loaded!")
        print(f"   Sequences: {len(self.X)}")
        print(f"   X shape: {self.X.shape}")
        print(f"   y shape: {self.y.shape}")
        print(f"   Class distribution: {np.bincount(self.y)}")
        print(f"{'='*60}\n")
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

In [20]:
def make_dataloader(dataset, batch_size=64, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=0,
        pin_memory=not GPU_MODE,
        prefetch_factor=None
    )


In [21]:
train_dataset = TimeSeriesDataset(ddf_train.persist())
train = make_dataloader(train_dataset)
dev_dataset = TimeSeriesDataset(ddf_dev.persist())
dev = make_dataloader(dev_dataset)

test_dataset = TimeSeriesDataset(ddf_test.persist())
test = make_dataloader(test_dataset)

2025-11-08 15:38:23,796 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3a041e670c42f75806d675cd5fe67a01 initialized by task ('shuffle-transfer-3a041e670c42f75806d675cd5fe67a01', 3) executed on worker tcp://127.0.0.1:39677



Loading TimeSeriesDataset (chunk size: 10000)...


2025-11-08 15:38:24,672 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3a041e670c42f75806d675cd5fe67a01 deactivated due to stimulus 'task-finished-1762616304.6713634'


	Found 125459 unique records

	Chunk 1/13: Records 0-10000
		Loaded 2400000 rows
		Record 21: 240 rows (expected 60)
		Record 32: 240 rows (expected 60)
		Record 40: 240 rows (expected 60)
		Record 49: 240 rows (expected 60)
		Record 77: 240 rows (expected 60)
		Total loaded: 10000 sequences

	Chunk 2/13: Records 10000-20000
		Loaded 2400000 rows
		Total loaded: 20000 sequences

	Chunk 3/13: Records 20000-30000
		Loaded 2400000 rows
		Total loaded: 30000 sequences

	Chunk 4/13: Records 30000-40000
		Loaded 2400000 rows
		Total loaded: 40000 sequences

	Chunk 5/13: Records 40000-50000
		Loaded 2400000 rows
		Total loaded: 50000 sequences

	Chunk 6/13: Records 50000-60000
		Loaded 2400000 rows
		Total loaded: 60000 sequences

	Chunk 7/13: Records 60000-70000
		Loaded 2400000 rows
		Total loaded: 70000 sequences

	Chunk 8/13: Records 70000-80000
		Loaded 2400000 rows
		Total loaded: 80000 sequences

	Chunk 9/13: Records 80000-90000
		Loaded 2400000 rows
		Total loaded: 90000 sequences

	C

2025-11-08 15:40:28,450 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8fb58ac3280f58db087a18cde1d71080 initialized by task ('shuffle-transfer-8fb58ac3280f58db087a18cde1d71080', 7) executed on worker tcp://127.0.0.1:39677


...

	Found 1276 unique records

	Chunk 1/1: Records 0-1276
		Loaded 306240 rows
		Record 24: 240 rows (expected 60)
		Record 282: 240 rows (expected 60)
		Record 700: 240 rows (expected 60)
		Record 754: 240 rows (expected 60)
		Record 983: 240 rows (expected 60)


2025-11-08 15:40:30,388 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1792e9be018f0b3fdcfed9be881b9570 initialized by task ('shuffle-transfer-1792e9be018f0b3fdcfed9be881b9570', 8) executed on worker tcp://127.0.0.1:45783


		Total loaded: 1276 sequences

Dataset loaded!
   Sequences: 1276
   X shape: (1276, 60, 11)
   y shape: (1276,)
   Class distribution: [1158  104   13    1]


Loading TimeSeriesDataset (chunk size: 10000)...


2025-11-08 15:40:30,799 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1792e9be018f0b3fdcfed9be881b9570 deactivated due to stimulus 'task-finished-1762616430.79894'


	Found 1271 unique records

	Chunk 1/1: Records 0-1271
		Loaded 305040 rows
		Record 8: 240 rows (expected 60)
		Record 93: 240 rows (expected 60)
		Record 136: 240 rows (expected 60)
		Record 285: 240 rows (expected 60)
		Record 301: 240 rows (expected 60)
		Total loaded: 1271 sequences

Dataset loaded!
   Sequences: 1271
   X shape: (1271, 60, 11)
   y shape: (1271,)
   Class distribution: [1118  129   23    1]



# Attentive RNN

In [22]:
class AttentiveRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, attention = False, bidirectional = False):
        super().__init__()

        self.hidden_size = hidden_size
        if attention in ["dot", "concat", None]:
            self.attention = attention
        else:
            raise ValueError("attention can only be 'dot' or 'concat'!!!")
        self.num_directions = 2 if bidirectional else 1

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True, bidirectional = bidirectional)
        self.concat = nn.Linear(hidden_size * self.num_directions, 1)

        self.fc = nn.Linear(hidden_size * self.num_directions, num_classes)
        
    def forward(self, X):
        # X shape: (batch_size, seq_len, input_size)
        batch_size, seq_len, input_size = X.shape
        rnn_out, h_n = self.rnn(X)
        # rnn_output: (batch_size, seq_len, hidden_size * num_directions)
        # h_n: (num_layers * num_directions, batch_size, hidden_size)
        score = None
        if self.attention == "concat":
            atten_input = rnn_out.reshape(-1, self.hidden_size * self.num_directions)     # (batch_size * seq-Len, hidden_size * num_directons)
            score = self.concat(atten_input).view(batch_size, -1)                         # (batch_size, seq_len)
        
        elif self.attention == "dot":
            
            if self.num_directions == 2:
                last_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                last_hidden = h_n[-1,:,:]
        
            query = last_hidden.unsqueeze(1)                                              # (batch_size, 1, hidden_size * num_directions)
            score = torch.bmm(query, rnn_out.transpose(1,2)).squeeze(1)                   # (batch_size, seq_len)
        
        context = None
        attn_weights = None
        
        if score is not None:
            attn_weights = F.softmax(score, dim=1)                                       # (batch_size, seq_len)
            context = torch.bmm(attn_weights.unsqueeze(1), rnn_out)                       # (batch_size, 1, hidden_size * num_directions)
            context = context.squeeze(1)                                                  # (batch_size, hidden_size * num_directions)
        else:
            if self.num_directions == 2:
                context = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                context = h_n[-1,:,:]

        output = self.fc(context)
        
        return output, attn_weights

# Attentive LSTM

In [23]:
class AttentiveLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, attention=None, bidirectional=False):
        super().__init__()

        self.hidden_size = hidden_size
        if attention in ["dot", "concat", None]:
            self.attention = attention
        else:
            raise ValueError("attention can only be 'dot', 'concat', or None!!!")
        
        self.num_directions = 2 if bidirectional else 1

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        
        if self.attention == "concat":
            self.attention_layer = nn.Linear(hidden_size * self.num_directions, 1)

        self.fc = nn.Linear(hidden_size * self.num_directions, num_classes)
        
    def forward(self, X):
        # X shape: (batch_size, seq_len, input_size)
        batch_size, seq_len, _ = X.shape
        
        lstm_out, (h_n, c_n) = self.lstm(X)
        # lstm_out: (batch_size, seq_len, hidden_size * num_directions)
        # h_n: (num_layers * num_directions, batch_size, hidden_size)
        # c_n: (num_layers * num_directions, batch_size, hidden_size)
        
        score = None
        if self.attention == "concat":
            atten_input = lstm_out.reshape(-1, self.hidden_size * self.num_directions)
            score = self.attention_layer(atten_input).view(batch_size, -1)
        
        elif self.attention == "dot":
            if self.num_directions == 2:
                last_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                last_hidden = h_n[-1,:,:]
        
            query = last_hidden.unsqueeze(1)
            score = torch.bmm(query, lstm_out.transpose(1,2)).squeeze(1)
        
        context = None
        attn_weights = None
        
        if self.attention is not None:
            attn_weights = F.softmax(score, dim=1)
            context = torch.bmm(attn_weights.unsqueeze(1), lstm_out)
            context = context.squeeze(1)
        else:
            if self.num_directions == 2:
                context = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                context = h_n[-1,:,:]

        output = self.fc(context)
        
        return output, attn_weights

# Training and Evaluation Functions

In [24]:
def training(model, device, criterion, optimizer, train_loader, train_epochs=10, pre_epochs=0):
    print(f"Starting training for {train_epochs} epochs on {device}.")
    train_start = time.time()
    
    for epoch in range(train_epochs):
        model.train()
        total_loss = 0.0
        all_predictions = []
        all_targets = []
        
        pbar = tqdm(train_loader, desc=f"Epoch {pre_epochs + epoch + 1}/{pre_epochs + train_epochs}")
        
        for features, labels in pbar:
            features = features.to(device) 
            labels = labels.to(device)

            optimizer.zero_grad()
            logits, _ = model(features)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted_classes = torch.max(logits, 1)
            all_predictions.extend(predicted_classes.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
            
            pbar.set_postfix(loss=f"{loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        accuracy = accuracy_score(all_targets, all_predictions)
        
        print(f"Epoch {pre_epochs + epoch + 1} Summary -> Avg. Loss: {avg_loss:.4f}, Accuracy: {accuracy*100:.2f}%")

    total_seconds = time.time() - train_start
    hours, rem = divmod(total_seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f"\nTotal Train Time: {int(hours)}h {int(minutes)}m {int(seconds)}s")

In [30]:
def eval_detailed(model, device, loader, report_title="Evaluation"):
    model.eval()
    all_targets, all_predictions, all_probas = [], [], []

    with torch.no_grad():
        for features, labels in tqdm(loader, desc=f"Evaluating ({report_title})", leave=False):
            features, labels = features.to(device), labels.to(device)
            logits, _ = model(features)
            probas = F.softmax(logits, dim=1)
            preds = torch.argmax(probas, dim=1)

            all_targets.extend(labels.cpu().numpy())
            all_predictions.extend(preds.cpu().numpy())
            all_probas.extend(probas.cpu().numpy())

    y_true = np.array(all_targets)
    y_pred = np.array(all_predictions)
    y_pred_proba = np.array(all_probas)

    print(f"\n================= {report_title} =================")
    print("Classification Report:\n")
    print(classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        digits=4,
        zero_division=0
    ))

    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
    except ValueError:
        roc_auc = float("nan")

    metrics = {
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "roc_auc_weighted": roc_auc,
        "mcc": matthews_corrcoef(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average='macro', zero_division=0),
        "f1_weighted": f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }

    cm = confusion_matrix(y_true, y_pred, normalize='true')
    print("\nNormalized Confusion Matrix:\n", cm)

    print("\nKey Metrics Summary:")
    for k, v in metrics.items():
        print(f"  {k:20s}: {v:.4f}")

    return metrics


In [31]:
def save_model_as_zip(model, model_name, save_dir="/kaggle/working"):
    # File paths
    model_path = os.path.join(save_dir, f"{model_name}.pth")
    zip_path = os.path.join(save_dir, f"{model_name}.zip")

    # Save model
    torch.save(model.state_dict(), model_path)

    # Zip it
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_path, arcname=f"{model_name}.pth")

    # Remove unzipped .pth file
    os.remove(model_path)

    # Get zipped size in MB
    size_bytes = os.path.getsize(zip_path)
    size_mb = size_bytes / (1024 ** 2)

    print(f"{model_name}.zip saved at {zip_path} | Size: {size_mb:.2f} MB")

# Experimenter

In [32]:
experiments = []
model_types = ["RNN", "LSTM"]
attention_types = [None, "dot", "concat"]
directionality = [False, True]

for model_t in model_types:
    for att_t in attention_types:
        for is_bi in directionality:
            # Create a descriptive name for the run
            direction_str = "Bi" if is_bi else "Uni"
            attention_str = att_t if att_t is not None else "NoAtt"
            run_name = f"{model_t}_{direction_str}_{attention_str}"
            
            config = {
                "run_name": run_name,
                "model_type": model_t,
                "hidden_size": 128,
                "num_layers": 2,
                "attention": att_t,
                "bidirectional": is_bi,
                "lr": 0.001,
                "epochs": 60
            }
            experiments.append(config)

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    if config["model_type"] == "RNN":
        model = AttentiveRNN(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_layers=config["num_layers"],
            num_classes=len(class_names),
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    else: # LSTM
        model = AttentiveLSTM(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_classes=len(class_names),
            num_layers=config["num_layers"],
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "model_type": config["model_type"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "attention": str(config["attention"]),
        "bidirectional": config["bidirectional"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("experiment_report_final.csv", index=False)


============================== Experiment 1/12: RNN_Uni_NoAtt ==============================
Using 2 GPUs via DataParallel!
Training model: RNN_Uni_NoAtt for 60 epochs...
Starting training for 60 epochs on cuda.


Epoch 1/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 1 Summary -> Avg. Loss: 0.2606, Accuracy: 90.26%


Epoch 2/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 2 Summary -> Avg. Loss: 0.2516, Accuracy: 90.45%


...

Epoch 60/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 60 Summary -> Avg. Loss: 0.0661, Accuracy: 97.54%

Total Train Time: 0h 21m 44s
Evaluating model: LSTM_Bi_concat...


Evaluating (LSTM_Bi_concat):   0%|          | 0/20 [00:00<?, ?it/s]


================= LSTM_Bi_concat =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9694    0.9845    0.9769      1158
     C-Class     0.8049    0.6346    0.7097       104
     M-Class     0.6471    0.8462    0.7333        13
     X-Class     1.0000    1.0000    1.0000         1

    accuracy                         0.9545      1276
   macro avg     0.8553    0.8663    0.8550      1276
weighted avg     0.9527    0.9545    0.9526      1276


Normalized Confusion Matrix:
 [[0.98445596 0.01295337 0.00259067 0.        ]
 [0.33653846 0.63461538 0.02884615 0.        ]
 [0.07692308 0.07692308 0.84615385 0.        ]
 [0.         0.         0.         1.        ]]

Key Metrics Summary:
  balanced_accuracy   : 0.8663
  roc_auc_weighted    : 0.9619
  mcc                 : 0.7158
  f1_macro            : 0.8550
  f1_weighted         : 0.9526
LSTM_Bi_concat.zip saved at /kaggle/working/LSTM_Bi_concat.zip | Size: 1.92 MB

============

,run_name,model_type,num_layers,hidden_size,attention,bidirectional,balanced_accuracy,roc_auc_weighted,mcc,f1_macro,f1_weighted,training_time_min
0,LSTM_Bi_concat,LSTM,2,128,concat,True,0.8663,0.9619,0.7158,0.8550,0.9526,21.7400
1,LSTM_Uni_concat,LSTM,2,128,concat,False,0.8309,0.9480,0.6838,0.8381,0.9472,18.3230
2,LSTM_Bi_dot,LSTM,2,128,dot,True,0.8145,0.9558,0.6541,0.8188,0.9424,22.0898
3,LSTM_Uni_dot,LSTM,2,128,dot,False,0.7658,0.9558,0.6460,0.7925,0.9410,18.1998
4,LSTM_Bi_NoAtt,LSTM,2,128,None,True,0.8128,0.9403,0.6201,0.8142,0.9366,20.9979
5,LSTM_Uni_NoAtt,LSTM,2,128,None,False,0.7593,0.9479,0.5635,0.7714,0.9274,17.1451
6,RNN_Uni_concat,RNN,2,128,concat,False,0.6585,0.9379,0.5536,0.6940,0.9263,16.2911
7,RNN_Bi_dot,RNN,2,128,dot,True,0.4330,0.9183,0.4716,0.4624,0.9120,19.0071
8,RNN_Bi_NoAtt,RNN,2,128,None,True,0.3807,0.9129,0.4228,0.4071,0.9052,17.8674
9,RNN_Bi_concat,RNN,2,128,concat,True,0.4096,0.9080,0.4219,0.4302,0.9064,18.3651


# Train Funrther

In [ ]:
def unzip_and_load_model(model, model_name, save_dir="/kaggle/working"):
    zip_path = os.path.join(save_dir, f"{model_name}.zip")
    model_path = os.path.join(save_dir, f"{model_name}.pth")

    # Unzip the model
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(save_dir)

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found after extraction: {model_path}")

    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location='cpu'))

    os.remove(model_path)

    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1+20}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    if config["model_type"] == "RNN":
        model = AttentiveRNN(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_layers=config["num_layers"],
            num_classes=len(class_names),
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    else: # LSTM
        model = AttentiveLSTM(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_classes=len(class_names),
            num_layers=config["num_layers"],
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
        
    model = unzip_and_load_model(model, config['run_name'])
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "model_type": config["model_type"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "attention": str(config["attention"]),
        "bidirectional": config["bidirectional"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("experiment_report_final2.csv", index=False)

# Multi Head Attention Bi-LSTM

In [39]:
class MultiHeadBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, num_heads=4):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_heads = num_heads

        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        # Multi-Head Attention
        self.mha = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,
            num_heads=num_heads,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, X):
        # X: (batch_size, seq_len, input_size)
        lstm_out, (h_n, c_n) = self.lstm(X)
        # lstm_out: (batch_size, seq_len, hidden_size * num_directions)

        attn_out, attn_weights = self.mha(lstm_out, lstm_out, lstm_out)
        # attn_out: (batch_size, seq_len, hidden_size * num_directions)

        context = torch.mean(attn_out, dim=1)
        # context: (batch_size, hidden_size * num_directions)

        output = self.fc(context)

        return output, attn_weights


In [40]:
experiments = []

# Experiment 1
run_name = "MH_Bi_LSTM_hs256_l2_h4"
config = {
    "run_name": run_name,
    "hidden_size": 256,
    "num_layers": 2,
    "heads": 4,
    "lr": 0.001,
    "epochs": 60
}
experiments.append(config)

# Experiment 2
run_name = "MH_Bi_LSTM_hs128_l2_h8"
config = {
    "run_name": run_name,
    "hidden_size": 128,
    "num_layers": 2,
    "heads": 8,
    "lr": 0.001,
    "epochs": 60
}
experiments.append(config)

# Experiment 3
run_name = "MH_Bi_LSTM_hs256_l3_h4"
config = {
    "run_name": run_name,
    "hidden_size": 256,
    "num_layers": 3,
    "heads": 4,
    "lr": 0.001,
    "epochs": 60
}
experiments.append(config)

# Experiment 4
run_name = "MH_Bi_LSTM_hs128_l1_h8"
config = {
    "run_name": run_name,
    "hidden_size": 128,
    "num_layers": 1,
    "heads": 8,
    "lr": 0.001,
    "epochs": 60
}
experiments.append(config)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    
    model = MultiHeadBiLSTM(
        input_size=len(features),
        hidden_size=config["hidden_size"],
        num_classes=len(class_names),
        num_layers=config["num_layers"],
        num_heads=config["heads"]
    )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
        
    # model = unzip_and_load_model(model, config['run_name'])
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "num_heads": config["heads"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("multihead_experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("multihead_experiment_report.csv", index=False)


============================== Experiment 1/4: MH_Bi_LSTM_hs256_l2_h4 ==============================
Using 2 GPUs via DataParallel!
Training model: MH_Bi_LSTM_hs256_l2_h4 for 60 epochs...
Starting training for 60 epochs on cuda.


Epoch 1/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 1 Summary -> Avg. Loss: 0.2559, Accuracy: 90.34%


Epoch 2/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 2 Summary -> Avg. Loss: 0.2450, Accuracy: 90.63%


...

Epoch 60/60:   0%|          | 0/1961 [00:00<?, ?it/s]

Epoch 60 Summary -> Avg. Loss: 0.1191, Accuracy: 94.87%

Total Train Time: 0h 20m 27s
Evaluating model: MH_Bi_LSTM_hs128_l1_h8...


Evaluating (MH_Bi_LSTM_hs128_l1_h8):   0%|          | 0/20 [00:00<?, ?it/s]


================= MH_Bi_LSTM_hs128_l1_h8 =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9593    0.9767    0.9679      1158
     C-Class     0.6629    0.5673    0.6114       104
     M-Class     0.6250    0.3846    0.4762        13
     X-Class     0.0000    0.0000    0.0000         1

    accuracy                         0.9365      1276
   macro avg     0.5618    0.4822    0.5139      1276
weighted avg     0.9310    0.9365    0.9331      1276


Normalized Confusion Matrix:
 [[9.76683938e-01 2.24525043e-02 8.63557858e-04 0.00000000e+00]
 [4.23076923e-01 5.67307692e-01 9.61538462e-03 0.00000000e+00]
 [3.07692308e-01 3.07692308e-01 3.84615385e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]

Key Metrics Summary:
  balanced_accuracy   : 0.4822
  roc_auc_weighted    : 0.9431
  mcc                 : 0.5956
  f1_macro            : 0.5139
  f1_weighted         : 0.9331
MH_Bi_LSTM_hs128_l1_

,run_name,num_layers,hidden_size,num_heads,balanced_accuracy,roc_auc_weighted,mcc,f1_macro,f1_weighted,training_time_min
0,MH_Bi_LSTM_hs256_l3_h4,3,256,4,0.8508,0.9228,0.6990,0.8679,0.9499,62.0284
1,MH_Bi_LSTM_hs128_l2_h8,2,128,8,0.5541,0.9472,0.6466,0.5583,0.9413,24.3983
2,MH_Bi_LSTM_hs256_l2_h4,2,256,4,0.7708,0.9419,0.6174,0.8050,0.9369,44.0901
3,MH_Bi_LSTM_hs128_l1_h8,1,128,8,0.4822,0.9431,0.5956,0.5139,0.9331,20.4653
